In [2]:
import pandas as pd
import numpy as np
import os

In [2]:
df = pd.read_csv("/run/media/fourier/Data2/Pras/Vale/TN_Forecast/fredformer/dataset/pi.csv")

In [3]:
def load_and_save(category, filename, dataset, dataset_folder):
    temp = np.genfromtxt(os.path.join(dataset_folder, category, filename),
                         dtype=np.float64,
                         delimiter=',')
    print(dataset, category, filename, temp.shape)
    np.save(os.path.join(output_folder, f"SMD/{dataset}_{category}.npy"), temp)
    return temp.shape


def load_and_save2(category, filename, dataset, dataset_folder, shape):
    temp = np.zeros(shape)
    with open(os.path.join(dataset_folder, 'interpretation_label', filename), "r") as f:
        ls = f.readlines()
    for line in ls:
        pos, values = line.split(':')[0], line.split(':')[1].split(',')
        start, end, indx = int(pos.split('-')[0]), int(pos.split('-')[1]), [int(i) - 1 for i in values]
        temp[start - 1:end - 1, indx] = 1
    print(dataset, category, filename, temp.shape)
    np.save(os.path.join(output_folder, f"SMD/{dataset}_{category}.npy"), temp)


def wgn(a, snr):
    min_a, max_a = np.min(a, axis=0), np.max(a, axis=0)
    x = (a - min_a) / (max_a - min_a + 0.0001)
    batch_size, len_x = x.shape
    Ps = np.sum(np.power(x, 2)) / len_x
    Pn = Ps / (np.power(10, snr / 10))
    noise = np.random.randn(len_x) * np.sqrt(Pn)
    return noise/100


def normalize(a):
    a = a / np.maximum(np.absolute(a.max(axis=0)), np.absolute(a.min(axis=0)))
    return a / 2 + 0.5


def normalize2(a, min_a=None, max_a=None):
    if min_a is None: min_a, max_a = min(a), max(a)
    return (a - min_a) / (max_a - min_a) + wgn(a, 50), min_a, max_a


def normalize3(a, min_a=None, max_a=None):
    if min_a is None: min_a, max_a = np.min(a, axis=0), np.max(a, axis=0)
    return ((a - min_a) / (max_a - min_a + 0.0001)), min_a, max_a #  + wgn(a, 50)


def convertNumpy(df):
    x = df[df.columns[3:]].values[::10, :]
    return (x - x.min(0)) / (x.ptp(0) + 1e-4)

In [5]:
dataset_folder = 'data/CustomAWGN30ES15'
df_train = pd.read_csv(os.path.join(dataset_folder, 'train.csv'))
df_test = pd.read_csv(os.path.join(dataset_folder, 'test.csv'))
df_train, df_test = df_train.values[:, 1:], df_test.values[:, 1:]
_, min_a, max_a = normalize3(np.concatenate((df_train, df_test), axis=0))
train, _, _ = normalize3(df_train, min_a, max_a)
test, _, _ = normalize3(df_test, min_a, max_a)
labels = pd.read_csv(os.path.join(dataset_folder, 'test_label.csv'))
labels = labels.values[:, 1:]

folder = os.path.join("processed", "CustomAWGN30ES15")
os.makedirs(folder, exist_ok=True)

for file in ['train', 'test', 'labels']:
    np.save(os.path.join(folder, f'{file}.npy'), eval(file).astype('float64'))

In [15]:
train.shape

(1196841, 30)